In [12]:
import tabula
import pandas as pd
import os
import datetime
import numpy as np
import calendar
def getList(path) -> pd.DataFrame:
    df = tabula.read_pdf(File,lattice=True,pages="1")
    if len(df[0]) > len(df[1]):
        return df[0]
    else:
        return df[1]

In [16]:
def AnalyzeDf(df: pd.DataFrame,year,month) -> bool:
    drop = [i for i in range(0,8)]
    df = df.drop(drop,axis=0)
    col = [i for i in range(0,len(df.columns))]
    df.columns = col

    df_r = pd.DataFrame()
    df_r["日付"] = df[1]
    df_r["曜日"] = df[2]
    df_r["出勤形態"] = df[3]
    df_r["出勤定時"] = df[4]
    df_r["出勤時間"] = df[5]
    df_r["退勤時間"] = df[6]
    df_r["申請理由"] = df[7]
    df_r["就業時間"] = df[9]
    df_r["普通残業"] = df[11]
    df_r["深夜残業"] = df[12]
    df_r["私的居残り"] = df[19]
    # df_r["出勤"] = df[26]
    # df_r["年休"] = df[27]
    # df_r["代休"] = df[28]
    # df_r["看護休暇"] = df[29]
    # df_r["特別休暇"] = df[29]
    # df_r["欠勤"] = df[29]
    # df_r["他"] = df[29]
    days = calendar.monthrange(year,month)[1]
    drop = [i for i in range(days+8,days+8+5)]
    df_r = df_r.drop(drop,axis=0)
    df_r["夜中フラグ"] = df_r[~pd.isna(df_r["退勤時間"])]["退勤時間"].str.contains("翌")
    df_r["退勤時間"] = df_r[~pd.isna(df_r["退勤時間"])]["退勤時間"].str.replace("翌","")
    df_r["退勤時間"] = pd.to_datetime(df_r[~pd.isna(df_r["退勤時間"])]["退勤時間"])
    df_r["夜中フラグ"] = df_r["夜中フラグ"].fillna(False)
    df_r["出勤フラグ"] = df_r["出勤形態"] == "◎出勤"
    df_r["勤務時間"] = df_r["出勤定時"].apply(lambda x: pd.to_datetime('17:00') if x=="◎0800" else None)
    df_r["超過勤務"] = df_r["退勤時間"] - df_r["勤務時間"]
    df_r["超過勤務"] = df_r["超過勤務"].apply(lambda x:x+datetime.timedelta(days=1) if x < datetime.timedelta(days=0) else x)
    df_r["普通残業"] = df_r["普通残業"].apply(lambda x:None if x == np.nan else pd.to_datetime(x)-pd.to_datetime("0:00"))
    df_r["深夜残業"] = df_r["深夜残業"].apply(lambda x:None if x == np.nan else pd.to_datetime(x)-pd.to_datetime("0:00"))
    df_r["私的居残り"] = df_r["私的居残り"].apply(lambda x:None if x == np.nan else pd.to_datetime(x)-pd.to_datetime("0:00"))
    df_r["普通残業"].fillna(datetime.timedelta(days=0),inplace=True)
    df_r["深夜残業"].fillna(datetime.timedelta(days=0),inplace=True)
    df_r["私的居残り"].fillna(datetime.timedelta(days=0),inplace=True)
    df_r["Flag1"] = df_r["超過勤務"] - df_r["普通残業"] - df_r["深夜残業"] - df_r["私的居残り"]
    df_r["Flag1"] = df_r["Flag1"].apply(lambda x: True if x == datetime.timedelta(0) else False)
    return not False  in  df_r[df_r['出勤フラグ']]['Flag1'].values

In [17]:
File = r'.\2022\2022.06_出勤簿\PLT2\PLT22\板東.pdf'
os.path.isfile(File)
df = getList(File)
AnalyzeDf(df,2022,6)

Got stderr: 7 11, 2022 9:14:43 午後 org.apache.fontbox.ttf.CmapSubtable processSubtype14
警告: Format 14 cmap table is not supported and will be ignored



False